# Assignment 1

Extracting Time Series Properties of Glucose Levels in Artificial Pancreas

In [12]:
'''
Imports
'''
from datetime import datetime, time

In [13]:
'''
Parse the csv file and find earliest auto switch
'''
autoDate = ''
with open('InsulinData.csv', 'r') as f:    
    i = 0
    for line in f:
        i += 1
        if i == 1:
            continue
            
        arr = line.split(",")        
        if arr[16] == 'AUTO MODE ACTIVE PLGM OFF':
            autoDate = arr[1]

# print(autoDate)

In [14]:
'''
Collect time stamp + reading data
'''
timeReadingDict = {}
with open('CGMData.csv', 'r') as f:
    '''
    Using prev value if reading is missing
    '''
#     prev = 130
    i = 0
    for line in f:
        # skip headers
        i += 1
        if i == 1:
            continue
            
        arr = line.split(",")
        dateTime = "{0}-{1}".format(arr[1], arr[2])
        reading = 0
        if arr[30] == '':
#             reading = prev
            continue
        else:
            reading = int(arr[30])
#             prev = reading
            
        timeReadingDict[dateTime] = reading
        
# print(timeReadingDict)

In [15]:
''' 
Collect readings by date 
'''

dateReadingListDict = {}

for time in timeReadingDict:
    date, clock = time.split('-')
    if date not in dateReadingListDict:
        dateReadingListDict[date] = {}
    dateReadingListDict[date][clock] = timeReadingDict[time]

'''
Splitting dates into auto and manual
'''

autoDatesReadingListDict = {}
manDatesReadingListDict = {}

switchDate = datetime.strptime(autoDate, "%m/%d/%Y")
for date in dateReadingListDict:
    curr = datetime.strptime(date, "%m/%d/%Y")
    if curr >= switchDate:
        autoDatesReadingListDict[date] = dateReadingListDict[date]
    if curr <= switchDate:
        manDatesReadingListDict[date] = dateReadingListDict[date]

# print(len(dateReadingListDict['2/10/2018']))

285


In [16]:
'''
Create day and night partitions
Copy data from date reading into these partitions
'''
autoDayDict = {}
autoNightDict = {}
manDayDict = {}
manNightDict = {}
dayLimit = datetime.strptime("06:00:00", "%H:%M:%S")

for date in autoDatesReadingListDict:
    for time in autoDatesReadingListDict[date]:
        curr = datetime.strptime(time, "%H:%M:%S")
        if curr < dayLimit:
            if date not in autoNightDict:
                autoNightDict[date] = {}
            autoNightDict[date][time] = autoDatesReadingListDict[date][time]
        else:
            if date not in autoDayDict:
                autoDayDict[date] = {}
            autoDayDict[date][time] = autoDatesReadingListDict[date][time]
            
for date in manDatesReadingListDict:
    for time in manDatesReadingListDict[date]:
        curr = datetime.strptime(time, "%H:%M:%S")
        if curr < dayLimit:
            if date not in manNightDict:
                manNightDict[date] = {}
            manNightDict[date][time] = manDatesReadingListDict[date][time]
        else:
            if date not in manDayDict:
                manDayDict[date] = {}
            manDayDict[date][time] = manDatesReadingListDict[date][time]
            
# print(autoDayDict['2/10/2018'])

{'23:55:30': 77, '23:50:30': 79, '23:45:30': 79, '23:40:30': 73, '23:35:30': 69, '23:30:30': 67, '23:25:30': 68, '23:20:30': 70, '23:15:30': 67, '23:10:30': 66, '23:05:30': 65, '23:00:30': 66, '22:55:30': 66, '22:50:30': 63, '22:45:30': 62, '22:40:30': 61, '22:35:30': 59, '22:30:30': 60, '22:25:30': 71, '22:20:30': 76, '22:15:30': 78, '22:10:30': 80, '22:05:30': 83, '22:00:30': 87, '21:55:30': 89, '21:50:30': 95, '21:45:30': 106, '21:40:30': 114, '21:35:30': 136, '21:30:30': 155, '21:25:30': 163, '21:20:30': 169, '21:15:30': 162, '21:10:30': 158, '21:05:30': 153, '21:00:30': 146, '20:55:30': 140, '20:50:30': 127, '20:45:30': 116, '20:40:30': 110, '20:35:30': 106, '20:30:30': 108, '20:25:30': 115, '20:20:30': 121, '20:15:30': 128, '20:10:30': 132, '20:05:30': 133, '20:00:30': 126, '19:55:30': 100, '19:50:30': 82, '19:45:30': 70, '19:40:30': 65, '19:35:30': 60, '19:30:30': 59, '19:25:30': 59, '19:20:30': 61, '19:15:30': 60, '19:10:30': 63, '19:05:30': 65, '19:00:30': 64, '18:55:30': 61, 

In [17]:
'''
Calculate percentage for each metric for each date
Calculate mean of all percentages over date
Params:- Dict of date:{time:reading}
Return:- List of means of each metric
'''
def getMeanPercentageMetrics(in_data):
    hrgList = []
    hrgCrtList = []
    rngList = []
    rsList = []
    hog1List = []
    hog2List = []

    for date in in_data:
        hrgSum = 0
        hrgCrtSum = 0
        rngSum = 0
        rsSum = 0
        hog1Sum = 0
        hog2Sum = 0
        for reading in in_data[date].values():
            if reading > 180:
                hrgSum += 1
            if reading > 250:
                hrgCrtSum += 1
            if reading >= 70 and reading <= 180:
                rngSum += 1
            if reading >= 70 and reading <= 150:
                rsSum += 1
            if reading < 70:
                hog1Sum += 1
            if reading < 54:
                hog2Sum += 1
        # for getting %
        hrgList.append(hrgSum/2.88)
        hrgCrtList.append(hrgCrtSum/2.88)
        rngList.append(rngSum/2.88)
        rsList.append(rsSum/2.88)
        hog1List.append(hog1Sum/2.88)
        hog2List.append(hog2Sum/2.88)
    
    avgList = [0,0,0,0,0,0]

    for hrg in hrgList:
        avgList[0] += hrg
    avgList[0] /= len(hrgList)

    for hrgCrt in hrgCrtList:
        avgList[1] += hrgCrt
    avgList[1] /= len(hrgCrtList)

    for rng in rngList:
        avgList[2] += rng
    avgList[2] /= len(rngList)

    for rs in rsList:
        avgList[3] += rs
    avgList[3] /= len(rsList)

    for hog1 in hog1List:
        avgList[4] += hog1
    avgList[4] /= len(hog1List)

    for hog2 in hog2List:
        avgList[5] += hog2
    avgList[5] /= len(hog2List)
    
    return avgList

In [18]:
autoWholeAvgList = getMeanPercentageMetrics(autoDatesReadingListDict)
autoDayAvgList = getMeanPercentageMetrics(autoDayDict)
autoNightAvgList = getMeanPercentageMetrics(autoNightDict)
manWholeAvgList = getMeanPercentageMetrics(manDatesReadingListDict)
manDayAvgList = getMeanPercentageMetrics(manDayDict)
manNightAvgList = getMeanPercentageMetrics(manNightDict)

In [19]:
def convertToString(in_list):
    res = ''
    for elem in in_list:
        res += "{},".format(str(elem))
    return res[:-1]

In [20]:
'''
Write to Results.csv
'''
with open('Results.csv', 'w') as f:
    f.write(convertToString(manNightAvgList+manDayAvgList+manWholeAvgList))
    f.write('\n')
    f.write(convertToString(autoNightAvgList+autoDayAvgList+autoWholeAvgList))